### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time

import torch
import pytorch_tabnet

from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.callbacks import Callback

import mlflow

from sklearn.preprocessing import LabelEncoder

import ads
import oci
import os
import hashlib

from ads.dataset.factory import DatasetFactory

from utils_Class import ODSDataCatalog

In [2]:
# setting security: using Resource Principal
ads.set_auth(auth='resource_principal')

In [3]:
# define the data catalog object the user wants to access, in this format: Catalog ID, Asset Key, Namespace ID
catalogo = ODSDataCatalog("ocid1.datacatalog.oc1.eu-frankfurt-1.aaaaaaaap6lel4hqckltn7fvjnux42jepzohdktkceywlyrnnrwgolbupzhq"
                         ,"6adf4ea3-67d5-44a3-b4f7-19fbf303d539","0e4d60d9-d5b5-467f-89bb-22db63a3ee18")

#### Get hash code from the file in the data catalog and compare it with the actual hash code of the file in the Object Storage

In [11]:
# define the file the user wants to analyze
file_name = "cs-test.csv"

# md5 read from Catalog
try: #try to use the function 
    hash_value = catalogo.get_hash_from_catalog(name=file_name)
    print()
    print('File hash from Data Catalog is:', hash_value)
except ValueError: #exception occurs if the function can't find the key name
    print('\033[1m'+'There is no such file in this Data catalog!'+'\033[0m')

No value registered in the data catalog!

File hash from Data Catalog is: None


In [10]:
file_name = "cs-test.csv"
bucket='credit_scoring'

# compute md5 from the file in the Object Storage
md5_comput = catalogo.get_hash_from_file(file_name,bucket)
print()
print('MD5 hash of the file is: ', md5_comput)

try:
    assert (md5_comput == hash_value)
    print("MD5 hash check OK")
except AssertionError:
    print('MD5 hash check failed')

The dataset contains: 101503 records

MD5 hash of the file is:  a05a4cef3c3880d003653bf772702bbb
MD5 hash check failed


#### Get version number from the file in the data catalog

In [12]:
# define the file the user wants to analyze
file_name = "cs-test.csv"

# version read from Catalog
try: #try to use the function 
    version_number = catalogo.get_version_from_catalog(name=file_name)
    print()
    print('Version Number from Data Catalog is:', version_number)
except ValueError: #exception occurs if the function can't find the key name
    print('\033[1m'+'There is no such file in this Data catalog!'+'\033[0m')



Version Number from Data Catalog is: v1.1
